In [1]:
import pandas as pd
import numpy as np
from grouper import group_by_volume, group_by_time
from pyfolio.timeseries import perf_stats
from functools import partial
import sys
sys.path.append('/home/tomek/ib_tools')
from indicators import get_ATR, get_signals
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

/home/tomek/.virtualenvs/ib/lib/python3.7/site-packages/pyfolio/pos.py:27: UserWarning: Module "zipline.assets" not found; mutltipliers will not be applied to position notionals.
  'Module "zipline.assets" not found; mutltipliers will not be applied' +


In [2]:
contract = pd.read_pickle('data/minute_NQ_cont_non_active_included.pickle')
multiplier = 20
min_tick = 0.25

In [3]:
contract = contract.loc['20180202':]
contract

,open,high,low,close,volume,average,barCount
date,,,,,,,
2018-02-02 00:00:00,6954.00,6958.00,6953.50,6955.25,877,6955.025,362
2018-02-02 00:01:00,6955.25,6956.50,6952.50,6952.50,376,6955.150,201
2018-02-02 00:02:00,6952.50,6954.00,6951.25,6952.00,292,6952.525,141
2018-02-02 00:03:00,6951.75,6953.50,6951.00,6953.00,414,6952.250,140
2018-02-02 00:04:00,6952.75,6956.50,6952.75,6956.50,304,6954.275,123
...,...,...,...,...,...,...,...
2019-09-04 18:27:00,7695.50,7696.00,7694.75,7695.00,369,7695.475,178
2019-09-04 18:28:00,7695.00,7695.50,7694.75,7695.00,173,7695.025,72
2019-09-04 18:29:00,7695.00,7695.75,7694.50,7695.00,195,7695.125,87


In [4]:
#periods = 20
periods = [5, 10, 20, 40, 80] #, 160, 320]
ema_fast = 120 # number of periods for moving average filter
sl_atr = 1 # stop loss in ATRs
atr_periods = 80 # number of periods to calculate ATR on
time_int = 15 # interval in minutes to be used to define volume candle
commission = 2.1 # level of commision per transaction in dollars
bankroll = 15000

In [5]:
avg_vol = contract.loc['20190621':].volume.rolling(time_int).sum().mean()
avg_vol

4957.834123352981

In [6]:
avg_vol = 14914.0 

In [7]:
vol_candles = group_by_volume(contract, avg_vol)
vol_candles['atr'] = get_ATR(vol_candles, atr_periods)

In [8]:
vol_candles

,date,open,high,low,close,barCount,volume,atr
label,,,,,,,,
0,2018-02-02 02:42:00,6954.00,6963.50,6912.50,6915.00,7971,14857,51.000000
1,2018-02-02 07:59:00,6915.25,6944.00,6915.00,6932.75,8645,14991,39.862500
2,2018-02-02 10:20:00,6932.75,6939.75,6888.00,6889.50,7977,14427,43.924470
3,2018-02-02 12:04:00,6889.75,6889.75,6857.75,6867.25,9138,16018,40.830641
4,2018-02-02 13:34:00,6867.00,6872.50,6844.50,6866.25,8830,15101,38.134623
...,...,...,...,...,...,...,...,...
12029,2019-09-04 16:43:00,7675.75,7683.75,7668.00,7676.50,7942,15511,24.377828
12030,2019-09-04 17:08:00,7676.50,7686.75,7673.00,7684.75,7686,14993,24.115413
12031,2019-09-04 17:32:00,7685.00,7698.75,7679.50,7697.25,7165,15097,23.995279


In [9]:
data = vol_candles
data.loc[:, 'ema'] = data.close.ewm(span=ema_fast).mean()
#data.loc[:, 'min'] = data.close.rolling(periods).min()
#data.loc[:,'max'] = data.close.rolling(periods).max()
rows_to_drop = max(periods)
#rows_to_drop = 80
rows_to_drop = max(rows_to_drop, ema_fast)
data = data.iloc[rows_to_drop:]
data.reset_index(drop=True, inplace=True)

In [10]:
#data['buy_signal'] = (data.close >= data['max']) * 1
#data['sell_signal'] = (data.close <= data['min']) * -1
#data['signal'] = data.buy_signal + data.sell_signal
#del data['buy_signal']
#del data['sell_signal']
data['signal'] = get_signals(data.close, periods)
data['filter'] = np.sign(data['close'] - data['ema'])
data['filtered_signal'] = data['signal'] * ((data['signal'] * data['filter']) == 1)
#data['cum_position'] = data['position'].cumsum().clip(lower=-1, upper=1)
#data.tail(50)

In [11]:
# while in position maintain open price and transaction direction
data['positions'] = 0
# flag to execute transaction at next data point
data['mark'] = False
# note the reason for transaction at next data point
data['reason'] = ''
# record commission paid
data['commission'] = 0
# record transaction price
data['price'] = 0
# entry price for stop loss calculation
data['entry'] = 0

for item in data.itertuples(): 
    # first row doesn't have to check if we have positions or execute transactions
    if not item.Index == 0:
        # starting position is the same as previous day position
        data.loc[item.Index, 'positions'] = data.loc[(item.Index - 1), 'positions'] 
        data.loc[item.Index, 'entry'] = data.loc[(item.Index - 1), 'entry'] 
        # execute transactions
        if data.loc[(item.Index -  1), 'mark']:
            # close position
            if data.loc[item.Index, 'positions']:
                data.loc[item.Index, 'positions'] = 0
                data.loc[item.Index, 'entry'] = 0
                # record transaction price
                data.loc[item.Index, 'price'] = item.open * np.sign(data.loc[(item.Index - 1), 'entry']) * -1
            # open position
            else:
                data.loc[item.Index, 'positions'] = data.loc[(item.Index - 1), 'signal']
                data.loc[item.Index, 'entry'] = item.open * data.loc[(item.Index - 1), 'signal']
                # record transaction price
                data.loc[item.Index, 'price'] = item.open * data.loc[(item.Index -1), 'signal']
            # record commission paid
            data.loc[item.Index, 'commission'] = commission

    # check for close signal
    if data.loc[item.Index, 'positions'] != 0 and np.sign(item.signal) != 0:
        if np.sign(data.loc[item.Index, 'positions']) != np.sign(item.signal):
            data.loc[item.Index, 'mark'] = True
            data.loc[item.Index, 'reason'] = 'close'
    # check for stop-loss signal
    if data.loc[item.Index, 'positions'] != 0:
        if data.loc[item.Index, 'positions'] > 0:
            # long positions
            if item.close <= (data.loc[item.Index, 'entry'] - (item.atr * sl_atr)):
                data.loc[item.Index, 'mark'] = True
                data.loc[item.Index, 'reason'] = 'stop-out'
            # short positions
            elif item.close >= (data.loc[item.Index, 'entry'] + (item.atr * sl_atr)):
                data.loc[item.Index, 'mark'] = True
                data.loc[item.Index, 'reason'] = 'stop-out'
    # check for entry signal
    if data.loc[item.Index, 'positions'] == 0:
        if item.filtered_signal != 0:
            data.loc[item.Index, 'mark'] = True
            data.loc[item.Index, 'reason'] = 'entry'
# close any open positions
if data[data.price != 0].price.count() % 2 != 0:
    data.loc[data.index[-1]+1, 'price'] = data.open.iloc[-1] * np.sign(data.entry.iloc[-1]) * -1
    data.loc[data.index[-2]+1, 'entry'] = 0
data.set_index('date', inplace=True)

In [12]:
pnl = data[['open','positions', 'commission', 'mark', 'reason', 'price', 'entry']]
pnl['mark'] = pnl['mark'].shift(1)
pnl['reason'] = pnl['reason'].shift(1)
pnl['cum_pnl'] = pnl['price'].cumsum() - pnl['entry']
pnl['balance'] = bankroll + pnl.cum_pnl * multiplier

In [13]:
pnl

,open,positions,commission,mark,reason,price,entry,cum_pnl,balance
date,,,,,,,,,
2018-02-06 15:58:00,6592.50,0.0,0.0,NaN,NaN,0.0,0.0,0.00,15000.0
2018-02-06 16:04:00,6523.50,0.0,0.0,False,,0.0,0.0,0.00,15000.0
2018-02-06 16:12:00,6587.00,0.0,0.0,False,,0.0,0.0,0.00,15000.0
2018-02-06 16:19:00,6542.75,0.0,0.0,False,,0.0,0.0,0.00,15000.0
2018-02-06 16:25:00,6499.00,0.0,0.0,False,,0.0,0.0,0.00,15000.0
...,...,...,...,...,...,...,...,...,...
2019-09-04 16:43:00,7675.75,0.0,0.0,False,,0.0,0.0,-744.25,115.0
2019-09-04 17:08:00,7676.50,0.0,0.0,False,,0.0,0.0,-744.25,115.0
2019-09-04 17:32:00,7685.00,0.0,0.0,False,,0.0,0.0,-744.25,115.0


In [14]:
daily = pnl.resample('D').last()
daily['return'] = daily.balance.pct_change()

In [15]:
pnl.price.sum()

-744.25

In [16]:
pnl.price.sum() *multiplier

-14885.0

In [17]:
pnl.commission.sum()

2616.6000000000004

In [18]:
pnl.price.sum()*multiplier - pnl.commission.sum()

-17501.6

In [19]:
pnl[pnl['price'] != 0].price.count() / daily.price.count()

3.0390243902439025

In [20]:
perf_stats(daily['return'])

Annual return          -0.881282
Cumulative returns     -0.992333
Annual volatility      72.289169
Sharpe ratio           -0.345924
Calmar ratio           -0.638919
Stability                    NaN
Max drawdown           -1.379333
Omega ratio             0.751928
Sortino ratio          -0.376384
Skew                         NaN
Kurtosis                     NaN
Tail ratio              0.706265
Daily value at risk    -9.206812
dtype: float64

In [21]:
daily.price.count()

410

In [22]:
pnl[pnl.price != 0]

,open,positions,commission,mark,reason,price,entry,cum_pnl,balance
date,,,,,,,,,
2018-02-06 21:12:00,6627.00,1.0,2.1,True,entry,6627.00,6627.00,0.00,15000.0
2018-02-07 16:15:00,6679.25,0.0,2.1,True,stop-out,-6679.25,0.00,-52.25,13955.0
2018-02-07 16:25:00,6684.75,1.0,2.1,True,entry,6684.75,6684.75,-52.25,13955.0
2018-02-07 19:09:00,6640.75,0.0,2.1,True,stop-out,-6640.75,0.00,-8.25,14835.0
2018-02-07 19:29:00,6611.25,-1.0,2.1,True,entry,-6611.25,-6611.25,-8.25,14835.0
...,...,...,...,...,...,...,...,...,...
2019-08-30 16:34:00,7713.75,0.0,2.1,True,stop-out,-7713.75,0.00,-725.50,490.0
2019-09-02 04:49:00,7630.25,-1.0,2.1,True,entry,-7630.25,-7630.25,-725.50,490.0
2019-09-04 07:33:00,7655.00,0.0,2.1,True,close,7655.00,0.00,-700.75,985.0


In [23]:
pnl[pnl.price != 0].count()

open          1246
positions     1246
commission    1246
mark          1246
reason        1246
price         1246
entry         1246
cum_pnl       1246
balance       1246
dtype: int64